<a href="https://colab.research.google.com/github/navneetkrc/Open_LLM_Apps/blob/main/QA_Generations/Streamlit_App_to_Generate_QA_Pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Streamlit App to Generate 16 types of QA Pairs
Also save as mardown format and txt format


# 📚 Support Doc Q\&A Generator

This project is designed to process Samsung support documents, generate various types of Q\&A pairs (e.g., troubleshooting, FAQs, setup guides), and provide the results in multiple formats (Markdown, Text).

---

## **1. prompts_util.py**

The `prompts_util.py` file contains utility functions for generating prompts for different types of Q\&A pairs. It organizes prompts into a dictionary and provides a single function (`get_prompt`) to fetch the appropriate prompt based on the selected Q\&A type.

### **Functions:**

- **`get_prompt(qa_type, text)`**:
    - Fetches the appropriate prompt for the given Q\&A type.
    - Combines the prompt template with the provided document text.
    - Raises an error if an unknown Q\&A type is requested.


### **Supported Q\&A Types:**

1. **Simple Q\&A**: General informative pairs covering key features, basic troubleshooting, and usage guidelines.
2. **Problem-Solution**: Focuses on problem descriptions and step-by-step solutions with severity indicators.
3. **Step-by-Step**: Extracts procedural guides with sequential steps, tool requirements, and time estimates.
4. **FAQ**: Predicts common user questions (e.g., "Why," "How," "What if").
5. **Troubleshooting**: Includes error symptoms, diagnostic checks, and repair procedures with priority levels.
6. **Feature Explanation**: Explains technologies with specs, use cases, and compatibility notes.
7. **Setup/Installation**: Covers pre-installation checks, configuration options, and post-setup tests.
8. **Error Codes**: Explains error codes with interpretations, immediate actions, and service triggers.
9. **Safety**: Generates safety-related Q\&A covering hazard prevention and compliance standards.
10. **Compatibility**: Covers device compatibility with models, version requirements, and regional variations.
11. **Maintenance**: Preventive maintenance tasks including cleaning procedures and part replacements.
12. **Warranty**: Covers warranty details, claim processes, and void conditions.
13. **Software Updates**: Includes version histories, rollback methods, and security patches.
14. **Preventive Care**: Best practices for failure prevention with early warning signs.
15. **Accessory Integration**: Covers pairing steps, compatibility checks, and optimal usage of accessories.
16. **Legal/Compliance**: Focuses on certifications, disposal guidelines, and privacy compliance.

---

## **2. app.py**

The `app.py` file is a Streamlit-based application that provides a user interface for generating Q\&A pairs from Samsung support documents.

### **Features:**

1. **Input Fields**:
    - URL input for the support document PDF.
    - Optional input for the Gemini API key (uses system default if left blank).
2. **Q\&A Type Selection**:
    - Users can select from 16 types of Q\&A pairs using checkboxes.
    - A "Select All" option is available to choose all types at once.
3. **Processing Workflow**:
    - Downloads the PDF from the provided URL.
    - Converts the PDF to Markdown format using `pymupdf4llm`.
    - Sends prompts one by one to the Gemini model to generate Q\&A pairs.
4. **Results Display \& Download**:
    - Displays generated Q\&A pairs in expandable sections for each selected type.
    - Provides download options for aggregated results in Markdown format.

### **Code Structure:**

#### Input Section

```python
pdf_url = st.text_input("🌐 Enter Support Document PDF URL")
api_key = st.text_input("🔑 Enter Gemini API Key (optional)", type="password")
```


#### Checkbox Selection

```python
qa_types = [list of 16 QA types]
select_all = st.checkbox("Select All", value=False)
selected_qa_types = []

cols = st.columns(4)
for idx, qa_type in enumerate(qa_types):
    col_idx = idx % 4
    checked = cols[col_idx].checkbox(qa_type, value=(select_all or idx < 5))
    if checked:
        selected_qa_types.append(qa_type)
```


#### Processing Workflow

```python
if pdf_url and st.button("🚀 Generate Q&A"):
    pdf_response = requests.get(pdf_url)
    markdown_text = pymupdf4llm.to_markdown("support_doc.pdf")
    results = {}
    for qa_type in selected_qa_types:
        prompt = get_prompt(qa_type, markdown_text)
        response_text = genai.GenerativeModel('gemini-2.0-flash').generate_content(prompt).text
        results[qa_type] = response_text
        st.expander(f"{qa_type} Results").markdown(response_text)
```


#### Download Options

```python
aggregated_markdown_content = ""
for qa_type, result in results.items():
    aggregated_markdown_content += f"## {qa_type}\n\n{result}\n\n---\n\n"

st.download_button(
    label="📥 Download Aggregated Results (Markdown)",
    data=aggregated_markdown_content,
    file_name="aggregated_results.md"
)
```

---

## Example Workflow

1. Enter a Samsung support document URL (e.g., `https://downloadcenter.samsung.com/...`).
2. Optionally enter your Gemini API key or use the default system key.
3. Select desired Q\&A types using checkboxes or click "Select All."
4. Click "Generate Q\&A" to process the document and generate results.
5. View results in expandable sections or download them as a Markdown file.

---

## Summary Table

| File | Purpose |
| :-- | :-- |
| `prompts_util.py` | Contains prompt templates for generating different types of Q\&A pairs based on user input. |
| `app.py` | Streamlit app that provides a user interface for generating and downloading Q\&A pairs |

This implementation ensures modularity (`prompts_util.py`) while providing an intuitive UI (`app.py`) for end-users to interact with Samsung support documents effectively!

<div style="text-align: center">⁂</div>

[^1]: https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/55174337/b45037b2-7c47-4793-b034-8d7a6e2b97be/qa_generation_for_kitchen_appliances_from_support_docs.py

[^2]: https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/55174337/1f2f058d-3c07-4a39-879d-e77a4ede61ee/Samsung-Support-Doc-Q-A-Generator.pdf



In [ ]:
#Step 1: Install Dependencies and Setup ngrok

# Install required packages
!pip install streamlit pyngrok pymupdf4llm fpdf google-generativeai -q

In [ ]:
#Step 1: Install Dependencies and Setup ngrok

# Import ngrok and set up authentication
from pyngrok import ngrok
from google.colab import userdata

# Set up ngrok authentication
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
if ngrok_token:
    !ngrok config add-authtoken {ngrok_token}
else:
    print("WARNING: No ngrok auth token found in Colab userdata. Public URL may not work correctly.")

In [7]:
%%writefile prompts_util.py
# prompts_util.py
from textwrap import dedent

# Define the prompts dictionary
prompts = {
    "Simple Q&A": """Generate fundamental Q&A pairs covering key features, basic troubleshooting, and usage guidelines.""",
    "Problem-Solution": """Create problem-solution pairs with severity indicators.""",
    "Step-by-Step": """Generate step-by-step procedural guides with tools and time estimates.""",
    "FAQ": """Predict common FAQs including 'Why', 'How', 'What if' scenarios.""",
    "Troubleshooting": """Create advanced troubleshooting guides with error symptoms, diagnostics, and priority levels.""",
    "Feature Explanation": """Explain Samsung technologies including technical specs, use cases, compatibility notes.""",
    "Setup/Installation": """Generate setup-related Q&A including pre-installation checks, configurations, post-setup tests.""",
    "Error Codes": """Explain error codes clearly with immediate actions and service triggers.""",
    "Safety": """Generate safety Q&A covering hazard prevention, compliance standards, recall info.""",
    "Compatibility": """Generate compatibility Q&A pairs covering supported models, versions, regional variations.""",
    "Maintenance": """Generate preventive maintenance Q&A including service intervals, cleaning procedures, part replacements.""",
    "Warranty": """Generate warranty Q&A covering coverage details, claim processes, void conditions.""",
    "Software Updates": """Generate software update Q&A including version histories, rollback methods, security patches.""",
    "Preventive Care": """Generate preventive care Q&A pairs including best practices, early warnings, monitoring tips.""",
    "Accessory Integration": """Generate accessory integration Q&A covering pairing steps, compatibility checks, optimal usage.""",
    "Legal/Compliance": """Generate regulatory/compliance Q&A pairs covering certification details, disposal guidelines, privacy compliance."""
}

# Function to fetch prompt based on QA type
def get_prompt(qa_type, text):
    if qa_type not in prompts:
        raise ValueError(f"Unknown QA type: {qa_type}")

    return dedent(f"{prompts[qa_type]}\n\nText: {text}")

Overwriting prompts_util.py


In [15]:
%%writefile streamlit_app.py
# import streamlit as st
import streamlit as st
import requests
import os
import pymupdf4llm
import google.generativeai as genai
from prompts_util import get_prompt  # Import prompt generation function

st.set_page_config(page_title="Support Doc Q&A Generator", layout="wide")
# st.set_page_config(page_title="Samsung Support Doc Q&A Generator", layout="wide")

st.title("📚 Support Doc Q&A Generator")
# st.title("📚 Samsung Support Doc Q&A Generator")

# URL input at top
pdf_url = st.text_input("🌐 Enter Support Document PDF URL")

# Optional API Key input below URL field
api_key = st.text_input("🔑 Enter Gemini API Key (optional)", type="password")
if not api_key:
    api_key = os.getenv('GOOGLE_API_KEY')
if not api_key:
    st.error("No default API key found. Please enter your Gemini API Key.")
    st.stop()

genai.configure(api_key=api_key)

# List of QA types
qa_types = [
    "Simple Q&A",
    "Problem-Solution",
    "Step-by-Step",
    "FAQ",
    "Troubleshooting",
    "Feature Explanation",
    "Setup/Installation",
    "Error Codes",
    "Safety",
    "Compatibility",
    "Maintenance",
    "Warranty",
    "Software Updates",
    "Preventive Care",
    "Accessory Integration",
    "Legal/Compliance"
]

# Checkbox selection for QA types
st.write("### Select Types of Q&A to Generate:")
select_all = st.checkbox("Select All", value=False)

selected_qa_types = []
cols = st.columns(4)  # Split checkboxes into 4 columns for better visibility

for idx, qa_type in enumerate(qa_types):
    col_idx = idx % 4  # Distribute checkboxes across columns
    checked = cols[col_idx].checkbox(qa_type, value=(select_all or idx < 5))  # Default select first 5 or all if 'Select All' is checked
    if checked:
        selected_qa_types.append(qa_type)

if pdf_url and st.button("🚀 Generate Q&A"):
    with st.spinner("Downloading PDF..."):
        pdf_response = requests.get(pdf_url)
        with open("support_doc.pdf", 'wb') as file:
            file.write(pdf_response.content)

    with st.spinner("Extracting Text..."):
        markdown_text = pymupdf4llm.to_markdown("support_doc.pdf")

    results = {}

    # Generate Q&A pairs for each selected type
    for qa_type in selected_qa_types:
        with st.spinner(f"Generating {qa_type}..."):
            prompt = get_prompt(qa_type, markdown_text)  # Get prompt for the QA type
            response_text = genai.GenerativeModel('gemini-2.0-flash').generate_content(prompt).text
            results[qa_type] = response_text

            # Display each generated QA type clearly in expandable sections
            with st.expander(f"{qa_type} Results"):
                st.markdown(response_text)

    # Aggregate markdown dynamically after loop completes fully
    aggregated_markdown_content = ""

    for qa_type, result in results.items():
        aggregated_markdown_content += f"## {qa_type}\n\n{result}\n\n---\n\n"

    # Download button for aggregated markdown clearly provided at end after loop completes fully
    st.download_button(
        label="📥 Download Aggregated Results (Markdown)",
        data=aggregated_markdown_content,
        file_name="aggregated_results.md"
     )


Overwriting streamlit_app.py


In [16]:
# Step 3.1: Close Existing ngrok Tunnels
#Close all existing tunnels to avoid limit errors
ngrok.kill()
print("✅ All existing ngrok tunnels closed successfully.")

✅ All existing ngrok tunnels closed successfully.


In [17]:
#Step 3.2: Run the Streamlit App and Create Tunnel

import subprocess, time
from pyngrok import ngrok

# Set environment variable for API key
import os
from google.colab import userdata

# Get GOOGLE_API_KEY from userdata
google_api_key = userdata.get('GOOGLE_API_KEY')
if google_api_key:
    os.environ['GOOGLE_API_KEY'] = google_api_key
else:
    print("Warning: No GOOGLE_API_KEY found in userdata.")

# Run Streamlit app in background on port 8501 explicitly set
subprocess.Popen(["streamlit", "run", "streamlit_app.py", "--server.port=8501"])

# Wait briefly for Streamlit to start before creating tunnel
time.sleep(5)

# Create ngrok tunnel explicitly on port 8501 and display public URL clearly
public_url = ngrok.connect(addr=8501)
print(f"\n✅ Streamlit app is running successfully!")
print(f"🌐 Public URL: {public_url}\n")



✅ Streamlit app is running successfully!
🌐 Public URL: NgrokTunnel: "https://35e1-35-237-9-124.ngrok-free.app" -> "http://localhost:8501"



#Streamlit App to Generate QA Pairs
Also save as mardown format and txt format

## Key Features of the Modified App

### Multiple Download Formats:
- **Text format (.txt)** - Simple plain text output  
- **Markdown format (.md)** - Formatted with headers and structure  

### Simplified Design:
- URL input field at the top  
- Optional API key field below  
- Clear success/error messages  

### Streamlined Process:
- Downloads PDF from provided URL  
- Extracts text using `pymupdf4llm`  
- Generates Q&A pairs using Gemini  
- Provides download options in multiple formats  

### User-Friendly Interface:
- Expandable sections for each Q&A category  
- Clear download buttons  
- Progress indicators during processing  

### Error Handling:
- Validates URL and PDF download  
- Handles API key issues gracefully  
- Provides clear error messages  

---

## How to Use the App  
1. Run the code cells in order in Google Colab  
2. Open the provided public URL  
3. Enter a Samsung support document URL  
4. Optionally provide your own Gemini API key  
5. Click **"Generate Q&A"**  
6. View the generated Q&A pairs in the expandable sections  
7. Download the results in your preferred format (text or markdown)  

> *This implementation meets all requirements while providing flexible download options for generated Q&A pairs.*  

In [13]:
#Step 1: Install Dependencies and Setup ngrok

# Install required packages
!pip install streamlit pyngrok pymupdf4llm fpdf google-generativeai -q

# Import ngrok and set up authentication
from pyngrok import ngrok
from google.colab import userdata

# Set up ngrok authentication
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
if ngrok_token:
    !ngrok config add-authtoken {ngrok_token}
else:
    print("WARNING: No ngrok auth token found in Colab userdata. Public URL may not work correctly.")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [15]:
#Step 2: Create the Streamlit App

%%writefile app.py
import streamlit as st
import requests
import os
import pymupdf4llm
import google.generativeai as genai
import base64
from textwrap import dedent
import json
import markdown

st.set_page_config(page_title="Samsung Support Doc Q&A Generator", layout="wide")

# Function to download PDF from URL
def download_pdf(url, filename="support_doc.pdf"):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(filename, 'wb') as f:
            f.write(response.content)
        return True, filename
    except Exception as e:
        return False, str(e)

# Function to convert PDF to markdown
def pdf_to_markdown(pdf_path):
    try:
        markdown_text = pymupdf4llm.to_markdown(pdf_path)
        return True, markdown_text
    except Exception as e:
        return False, str(e)

# Function to create a download link for text file
def get_download_link(text, filename="results.txt", label="Download Results"):
    b64 = base64.b64encode(text.encode()).decode()
    href = f'<a href="data:file/txt;base64,{b64}" download="{filename}">{label}</a>'
    return href

# Function to create a download link for markdown file
def get_markdown_download_link(text, filename="results.md", label="Download as Markdown"):
    b64 = base64.b64encode(text.encode()).decode()
    href = f'<a href="data:text/markdown;base64,{b64}" download="{filename}">{label}</a>'
    return href

# Function to generate Q&A pairs
def generate_qa_pairs(text, model_name='gemini-2.0-flash'):
    model = genai.GenerativeModel(model_name)

    # Define QA generation functions
    qa_functions = {
        "Simple Q&A": {
            "function": lambda text: model.generate_content(dedent(f"""
                Generate fundamental Q&A pairs from Samsung support docs covering:
                - Key features
                - Basic troubleshooting
                - Usage guidelines
                Use consumer-friendly language with concise answers (1-2 sentences).
                Text: {text}
                """)).text
        },
        "Problem-Solution": {
            "function": lambda text: model.generate_content(dedent(f"""
                Create problem-solution pairs from Samsung docs. Format:
                Question: [Issue description]
                Answer: [Step-by-step fix]
                Include severity indicators (⚠️) for critical issues.
                Text: {text}
                """)).text
        },
        "Step-by-Step": {
            "function": lambda text: model.generate_content(dedent(f"""
                Extract step-by-step procedures and generate Q&A pairs with:
                - Sequential questions
                - Tool requirements
                - Time estimates
                Format steps with numbered sub-items.
                Text: {text}
                """)).text
        },
        "FAQ": {
            "function": lambda text: model.generate_content(dedent(f"""
                Predict common FAQs from Samsung users based on this text.
                Phrase questions as natural customer inquiries.
                Include 'Why', 'How', and 'What if' scenarios.
                Text: {text}
                """)).text
        },
        "Troubleshooting": {
            "function": lambda text: model.generate_content(dedent(f"""
                Create advanced troubleshooting pairs including:
                - Error symptoms
                - Diagnostic checks
                - Repair procedures
                Format solutions with priority levels (Critical/High/Medium).
                Text: {text}
                """)).text
        }
    }

    results = {}
    for name, qa_function in qa_functions.items():
        try:
            results[name] = qa_function["function"](text)
        except Exception as e:
            results[name] = f"Error generating {name} Q&A: {str(e)}"

    return results

# Function to save results to text file
def save_results_to_text(results, filename="qa_results.txt"):
    with open(filename, "w", encoding="utf-8") as f:
        for category, result in results.items():
            f.write(f"\n\n{'='*50}\n{category}\n{'='*50}\n\n")
            f.write(result)

    with open(filename, "r", encoding="utf-8") as f:
        return f.read()

# Function to save results to markdown file
def save_results_to_markdown(results, filename="qa_results.md"):
    with open(filename, "w", encoding="utf-8") as f:
        for category, result in results.items():
            f.write(f"\n\n## {category}\n\n")
            f.write(result)

    with open(filename, "r", encoding="utf-8") as f:
        return f.read()

# Main app
def main():
    st.title("📚 Samsung Support Doc Q&A Generator")

    # URL input first
    pdf_url = st.text_input("🌐 Enter Support Document PDF URL")

    # API Key input (optional, below URL field)
    api_key = st.text_input("🔑 Enter Gemini API Key (optional)", type="password")

    if not api_key:
        api_key = os.getenv('GOOGLE_API_KEY')
        if api_key:
            st.success("Using system default Gemini API key.")
        else:
            st.error("No default API key found. Please enter your Gemini API Key.")
            st.stop()

    # Configure Gemini
    genai.configure(api_key=api_key)

    if pdf_url and st.button("🚀 Generate Q&A"):
        with st.spinner("Downloading document..."):
            success, result = download_pdf(pdf_url)

            if not success:
                st.error(f"Failed to download PDF: {result}")
                return

            st.success(f"Document downloaded as {result}")

            # Convert PDF to markdown
            with st.spinner("Converting PDF to text..."):
                success, markdown_text = pdf_to_markdown(result)

                if not success:
                    st.error(f"Failed to convert PDF: {markdown_text}")
                    return

                st.success("PDF converted to text successfully")

                # Generate Q&A pairs
                with st.spinner("Generating Q&A pairs... This may take a few minutes."):
                    results = generate_qa_pairs(markdown_text)

                    # Display results in expandable sections
                    for category, result in results.items():
                        with st.expander(f"{category} Q&A"):
                            st.markdown(result)

                    # Save results in different formats
                    text_content = save_results_to_text(results)
                    markdown_content = save_results_to_markdown(results)

                    # Provide download links
                    st.markdown("### Download Results")
                    col1, col2 = st.columns(2)
                    with col1:
                        st.markdown(get_download_link(text_content, "samsung_qa_results.txt", "Download as Text"), unsafe_allow_html=True)
                        st.markdown(get_markdown_download_link(markdown_content, "samsung_qa_results.md", "Download as Markdown"), unsafe_allow_html=True)

if __name__ == "__main__":
    main()


Writing app.py


In [14]:
# Step 3.1: Close Existing ngrok Tunnels
#Close all existing tunnels to avoid limit errors
ngrok.kill()
print("✅ All existing ngrok tunnels closed successfully.")


✅ All existing ngrok tunnels closed successfully.


In [16]:
#Step 3.2: Run the Streamlit App and Create Tunnel

import subprocess, time
from pyngrok import ngrok

# Set environment variable for API key
import os
from google.colab import userdata

# Get GOOGLE_API_KEY from userdata
google_api_key = userdata.get('GOOGLE_API_KEY')
if google_api_key:
    os.environ['GOOGLE_API_KEY'] = google_api_key
else:
    print("Warning: No GOOGLE_API_KEY found in userdata.")

# Run Streamlit app in background on port 8501 explicitly set
subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501"])

# Wait briefly for Streamlit to start before creating tunnel
time.sleep(5)

# Create ngrok tunnel explicitly on port 8501 and display public URL clearly
public_url = ngrok.connect(addr=8501)
print(f"\n✅ Streamlit app is running successfully!")
print(f"🌐 Public URL: {public_url}\n")



✅ Streamlit app is running successfully!
🌐 Public URL: NgrokTunnel: "https://db79-34-59-149-194.ngrok-free.app" -> "http://localhost:8501"



#old

In [25]:
#▶️ Step 1: Install Dependencies & Setup ngrok Authentication
# Install dependencies
!pip install streamlit pyngrok pymupdf4llm fpdf google-generativeai -q

# Setup ngrok authentication
from google.colab import userdata
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
if ngrok_token:
    !ngrok config add-authtoken {ngrok_token}
else:
    raise Exception("Please set NGROK_AUTH_TOKEN in your Colab userdata.")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [26]:
#▶️ Step 2: Download Unicode Font for PDF Generation

# Download DejaVuSans font for Unicode support
!wget https://github.com/dejavu-fonts/dejavu-fonts/raw/master/ttf/DejaVuSans.ttf -O DejaVuSans.ttf


--2025-03-08 22:07:18--  https://github.com/dejavu-fonts/dejavu-fonts/raw/master/ttf/DejaVuSans.ttf
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-08 22:07:18 ERROR 404: Not Found.



In [27]:
#▶️ Step 3: Create Streamlit App (app.py)
%%writefile app.py
import streamlit as st
import requests, pymupdf4llm, google.generativeai as genai, base64
from google.colab import userdata
from fpdf import FPDF
from textwrap import dedent

st.set_page_config(page_title="Support Doc Q&A Generator", layout="wide")
st.title("📚 Samsung Support Doc Q&A Generator")

# API Key handling (allow blank submission)
api_key = st.text_input("🔑 Enter Gemini API Key (leave blank to use default)", type="password")
if not api_key:
    api_key = userdata.get('GOOGLE_API_KEY')
    if api_key:
        st.success("Using default Gemini API key from system.")
    else:
        st.error("No default API key found. Please enter your Gemini API Key.")
        st.stop()

genai.configure(api_key=api_key)

# PDF URL input from user
pdf_url = st.text_input("🌐 Enter Support Document PDF URL")

if pdf_url and st.button("🚀 Generate Q&A"):
    with st.spinner("Downloading PDF..."):
        pdf_response = requests.get(pdf_url)
        with open("support_doc.pdf", "wb") as f:
            f.write(pdf_response.content)
        st.success("✅ PDF downloaded successfully.")

    with st.spinner("Extracting text from PDF..."):
        markdown_text = pymupdf4llm.to_markdown("support_doc.pdf")
        st.success("✅ Text extracted successfully.")

    # Generate Q&A using Gemini model
    model = genai.GenerativeModel('gemini-2.0-flash')
    prompt = dedent(f"""
        Generate structured Q&A pairs covering key features, troubleshooting, and usage guidelines.
        Text: {markdown_text}
    """)
    with st.spinner("Generating Q&A pairs..."):
        result = model.generate_content(prompt).text

    # Save results into a nicely formatted PDF using Unicode font
    pdf = FPDF()
    pdf.add_page()
    pdf.add_font('DejaVu', '', 'DejaVuSans.ttf', uni=True)
    pdf.set_font('DejaVu', size=12)

    pdf.multi_cell(0, 8, result)
    pdf.output("aggregated_results.pdf")

    # Provide download link for aggregated results PDF
    with open("aggregated_results.pdf", "rb") as file:
        btn = st.download_button(
            label="📥 Download Aggregated Results PDF",
            data=file,
            file_name="aggregated_results.pdf",
            mime="application/pdf"
        )


Overwriting app.py


In [29]:
#✅ Step 4.1: Ensure All Existing Tunnels Are Closed (Important Fix)
from pyngrok import ngrok

# Close all existing ngrok tunnels
tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)

# Verify no tunnels remain active
if len(ngrok.get_tunnels()) == 0:
    print("✅ All existing ngrok tunnels closed successfully.")
else:
    print("⚠️ Some tunnels are still open. Consider restarting runtime.")


✅ All existing ngrok tunnels closed successfully.


In [30]:
#▶️ Step 4.2: Run Streamlit App in Background & Expose via ngrok
from pyngrok import ngrok
import subprocess, time

# Run Streamlit app in background on port 8501 explicitly set
subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501"])

# Wait briefly for Streamlit to start before creating tunnel
time.sleep(5)

# Create ngrok tunnel explicitly on port 8501 and display public URL clearly
public_url = ngrok.connect(addr=8501)
print(f"\n✅ Streamlit app is running successfully!")
print(f"🌐 Public URL: {public_url}\n")



✅ Streamlit app is running successfully!
🌐 Public URL: NgrokTunnel: "https://f86b-34-70-185-151.ngrok-free.app" -> "http://localhost:8501"

